# ForgeBoot / TriadBoot Fractal Multi-Core Simulation Notebook

This notebook prototypes the ForgeBoot (miner focus) and TriadBoot (simulation/art focus) orchestration using Mandelbrot branching to gate multi-core tasks and a Merkle (φ-chain) ledger seal.

Outline Implemented:
1. Configuration constants
2. Imports
3. Complex seed init
4. Mandelbrot iteration helper
5. Branch angles
6. Escape/prune logic
7. Core spin scheduler
8. SimEtch 48D flux kernel
9. ArtPipe spiral generator
10. Thermal drift monitor
11. Fractal branch visualizer (optional)
12. Merkle leaf hash
13. Merkle tree builder
14. Ledger assembly
15. JSON export
16. Performance benchmark & uptime
17. Main orchestrator


In [ ]:
# 1. Configuration Constants
phi = 1.618  # golden ratio
core_count = 8
nm_scale = 8e-9  # 8 nm gate scale conceptual
flux_target_rvn = 100_000_000_000  # 100 GW conceptual placeholder
flux_target_sim = 48  # dimension count placeholder
escape_radius = 2.0
max_depth = 5
branch_length_initial = 100.0  # for turtle viz (pixels)
drift_tol = 1e-12
uptime_target = 0.9999
r_hash_target = 2_000_000  # target hashes/sec conceptual

config = dict(phi=phi, core_count=core_count, nm_scale=nm_scale,
              escape_radius=escape_radius, max_depth=max_depth,
              drift_tol=drift_tol, uptime_target=uptime_target,
              r_hash_target=r_hash_target)
config

In [ ]:
# 2. Import Libraries
import math, cmath, time, json, hashlib, statistics, concurrent.futures
try:
    import turtle  # optional visualizer
    TURTLE_AVAILABLE = True
except Exception:
    TURTLE_AVAILABLE = False
import numpy as np

print("Turtle available?", TURTLE_AVAILABLE)

### Mandelbrot Formula
We use $z_{k+1} = z_k^2 + c$ with complex seed values:
- Initial: z0 = 0 + 0j
- Constant: c = 0.3 + 0j

In [ ]:
# 3. Complex Seed Initialization
z0 = 0 + 0j
c = 0.3 + 0j
seed_config = dict(z0=str(z0), c=str(c))
seed_config

In [ ]:
# 4. Mandelbrot Iteration Helper

def iterate(z: complex, c: complex):
    """Return next z and magnitude."""
    nz = z*z + c
    return nz, abs(nz)

# quick test
z_test, mag_test = iterate(z0, c)
print(z_test, mag_test)

In [ ]:
# 5. Branch Angle Computation
import math

def branch_angles(z: complex):
    up_deg = 25.0 + abs(z.imag)*10.0
    down_deg = 50.0 + abs(z.imag)*20.0
    return math.radians(up_deg), math.radians(down_deg), up_deg, down_deg

print(branch_angles(0.3j))

In [ ]:
# 6. Escape / Prune Logic

def should_prune(z: complex, radius: float = escape_radius) -> bool:
    return abs(z) > radius

print(should_prune(3+0j), should_prune(0.5+0.5j))

In [ ]:
# 7. Hypercube Core Spin Scheduler
from concurrent.futures import ThreadPoolExecutor, as_completed

core_roles = ["GPU" if i % 2 == 0 else "CPU" for i in range(core_count)]

def core_task(core_id:int, steps:int=200):
    z = z0
    pruned = 0
    for _ in range(steps):
        z, mag = iterate(z, c)
        if should_prune(z):
            pruned += 1
            z = z0  # reset after prune
    return {"core": core_id, "role": core_roles[core_id], "pruned": pruned}

# quick parallel run
with ThreadPoolExecutor(max_workers=core_count) as ex:
    futs = [ex.submit(core_task, i) for i in range(core_count)]
    results = [f.result() for f in as_completed(futs)]
results

In [ ]:
# 8. SimEtch Flux Kernel (48D placeholder)

def simetch_flux(samples:int=10):
    flux_vals = []
    for _ in range(samples):
        vec48 = np.random.rand(48)
        # synthetic flux metric: product of mean & std scaled
        flux = float(vec48.mean() * vec48.std() * 100)
        flux_vals.append(flux)
    return flux_vals

print(simetch_flux(3))

In [ ]:
# 9. ArtPipe Spiral Generator (θ = φ*i*ω*t mod 2π)

def spiral(theta_count:int=200, omega:float=1.0):
    pts = []
    start = time.time()
    for i in range(theta_count):
        t = (time.time() - start)
        theta = (phi * i * omega * t) % (2*math.pi)
        r = i / theta_count
        x = r * math.cos(theta)
        y = r * math.sin(theta)
        pts.append((x,y))
    return pts

spiral_pts = spiral(50)
spiral_pts[:5]

In [ ]:
# 10. Thermal Drift Monitor

def thermal_drift(series, target:float=0.0):
    if not series:
        return 0.0
    drift = abs(statistics.fmean(series) - target)
    return drift

# simulate drift check
drift_val = thermal_drift([0.0, 1e-13, -5e-13])
print("Drift", drift_val, "OK?", drift_val < drift_tol)

In [ ]:
# 11. Fractal Branch Turtle Visualizer (optional)

def draw_branches(depth:int, length:float, z:complex, c:complex):
    if not TURTLE_AVAILABLE:
        print("Turtle not available; skipping drawing.")
        return
    if depth <= 0:
        return
    nz, _ = iterate(z, c)
    up_rad, down_rad, up_deg, down_deg = branch_angles(nz)
    turtle.forward(length)
    turtle.left(up_deg)
    draw_branches(depth-1, length*0.7, nz, c)
    turtle.right(up_deg + down_deg)  # return to stem then down
    draw_branches(depth-1, length*0.7, nz, c)
    turtle.left(down_deg)  # restore heading
    turtle.backward(length)

# Not auto-run to avoid blocking environments
print("Visualizer ready (call draw_branches(max_depth, branch_length_initial, z0, c))")

In [ ]:
# 12. Merkle Leaf Hash Function (φ-chain)

def hash_leaf(data:dict) -> bytes:
    enc = json.dumps(data, sort_keys=True).encode()
    prefix = f"{phi:.6f}".encode()
    return hashlib.sha256(prefix + enc).digest()

print(hash_leaf({"a":1}).hex())

In [ ]:
# 13. Merkle Tree Builder

def build_merkle(leaves: list[bytes]):
    import time
    start = time.time()
    layer = leaves[:]
    if not layer:
        return b"", 0.0, 0
    hash_ops = 0
    while len(layer) > 1:
        next_layer = []
        for i in range(0, len(layer), 2):
            a = layer[i]
            b = layer[i+1] if i+1 < len(layer) else layer[i]
            h = hashlib.sha256(a + b).digest()
            next_layer.append(h)
            hash_ops += 1
        layer = next_layer
    elapsed = time.time() - start
    hps = hash_ops/elapsed if elapsed > 0 else 0
    return layer[0], hps, hash_ops

# quick test
root, hps, ops = build_merkle([hash_leaf({"i":i}) for i in range(10)])
print(root.hex(), hps, ops)

In [ ]:
# 14. Ledger Assembly
ledger_events = []

# example event injection
ledger_events.append({"ts": time.time(), "type": "init", "config": config})

print(len(ledger_events))

In [ ]:
# 15. JSON Export Helper

def export_ledger(path: str, root_hash: bytes, merkle_hps: float, hash_ops: int):
    data = {
        "config": config,
        "events": ledger_events,
        "root_hash": root_hash.hex(),
        "merkle_hashes_per_sec": merkle_hps,
        "merkle_hash_ops": hash_ops,
        "timestamp": time.time()
    }
    with open(path, 'w') as f:
        json.dump(data, f, indent=2)
    print("Ledger exported", path)

# not exporting yet

In [ ]:
# 16. Performance Benchmark & Uptime Estimator

def performance_summary(core_results, start_time, end_time, pruned_total:int):
    run_time = end_time - start_time
    fault_time = 0  # placeholder; would accumulate downtime
    uptime = (run_time - fault_time)/run_time if run_time > 0 else 0
    avg_pruned = statistics.fmean([r['pruned'] for r in core_results]) if core_results else 0
    return {
        "run_time_sec": run_time,
        "uptime_estimate": uptime,
        "avg_pruned_per_core": avg_pruned,
        "total_pruned": pruned_total
    }

# example placeholder
perf = performance_summary(results, time.time()-1, time.time(), sum(r['pruned'] for r in results))
perf

In [ ]:
# 17. Main ForgeBoot / TriadBoot Orchestrator

def orchestrate(cycles:int=5, core_steps:int=300):
    start = time.time()
    ledger_events.append({"ts": start, "type": "orchestrate_start"})
    # Core spin
    with ThreadPoolExecutor(max_workers=core_count) as ex:
        futs = [ex.submit(core_task, i, core_steps) for i in range(core_count)]
        core_results = [f.result() for f in as_completed(futs)]
    # SimEtch & ArtPipe cycles
    flux_samples = []
    spiral_samples = []
    for _ in range(cycles):
        flux_samples.extend(simetch_flux(3))
        spiral_samples.extend(spiral(20))
    drift_val = thermal_drift([p[0] for p in spiral_samples], 0.0)
    pruned_total = sum(r['pruned'] for r in core_results)
    ledger_events.append({"ts": time.time(), "type": "metrics", "flux": flux_samples[:5], "drift": drift_val})
    # Merkle seal
    leaves = [hash_leaf({"event_index": i, "type": e['type']}) for i, e in enumerate(ledger_events)]
    root_hash, hps, ops = build_merkle(leaves)
    perf = performance_summary(core_results, start, time.time(), pruned_total)
    ledger_events.append({"ts": time.time(), "type": "seal", "root": root_hash.hex(), "hps": hps})
    export_ledger("ledger.json", root_hash, hps, ops)
    return {"root_hash": root_hash.hex(), "hps": hps, "ops": ops, "perf": perf, "drift_ok": drift_val < drift_tol}

final_summary = orchestrate()
final_summary